In [1]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["USE_TF"] = "0"

In [2]:
import pandas as pd

In [3]:
books = pd.read_csv("books_with_categories.csv")

In [4]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "mps")
classifier("I love this!")

[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.00852868054062128},
  {'label': 'neutral', 'score': 0.005764597561210394},
  {'label': 'anger', 'score': 0.004419779870659113},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.00041385198710486293}]]

In [5]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [6]:
classifier(books['description'][0])

[[{'label': 'fear', 'score': 0.654841423034668},
  {'label': 'neutral', 'score': 0.16985176503658295},
  {'label': 'sadness', 'score': 0.11640895158052444},
  {'label': 'surprise', 'score': 0.020700611174106598},
  {'label': 'disgust', 'score': 0.019100740551948547},
  {'label': 'joy', 'score': 0.015161240473389626},
  {'label': 'anger', 'score': 0.003935147542506456}]]

In [7]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [8]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156341664493084},
 {'label': 'disgust', 'score': 0.0026284721679985523},
 {'label': 'fear', 'score': 0.06816218793392181},
 {'label': 'joy', 'score': 0.04794243723154068},
 {'label': 'neutral', 'score': 0.1403854936361313},
 {'label': 'sadness', 'score': 0.0021221600472927094},
 {'label': 'surprise', 'score': 0.7296029329299927}]

In [9]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [10]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [11]:
emotion_scores

{'anger': [0.06413353979587555,
  0.6126182675361633,
  0.06413353979587555,
  0.351483553647995,
  0.08141202479600906,
  0.23222504556179047,
  0.538184404373169,
  0.06413353979587555,
  0.30067020654678345,
  0.06413353979587555],
 'disgust': [0.27359041571617126,
  0.3482869267463684,
  0.1040065810084343,
  0.1507226675748825,
  0.18449504673480988,
  0.7271746397018433,
  0.1558549404144287,
  0.1040065810084343,
  0.27948081493377686,
  0.17792750895023346],
 'fear': [0.9281687140464783,
  0.9425276517868042,
  0.9723208546638489,
  0.3607066571712494,
  0.09504330903291702,
  0.051362741738557816,
  0.7474284172058105,
  0.404496431350708,
  0.9155242443084717,
  0.051362741738557816],
 'joy': [0.9327982664108276,
  0.7044217586517334,
  0.7672364115715027,
  0.2518806755542755,
  0.04056435078382492,
  0.043375905603170395,
  0.8725653886795044,
  0.04056435078382492,
  0.04056435078382492,
  0.04056435078382492],
 'sadness': [0.6462160348892212,
  0.8879396319389343,
  0.549

In [12]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

  4%|▎         | 184/5197 [00:17<07:39, 10.92it/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

100%|██████████| 5197/5197 [05:22<00:00, 16.09it/s]


In [13]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [14]:
books = pd.merge(books, emotions_df, on = "isbn13")

In [15]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273590,0.928169,0.932798,0.646216,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348287,0.942528,0.704422,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767236,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351484,0.150723,0.360707,0.251881,0.732686,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255169,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759455,0.951104,0.368110,0.078765


In [16]:
books.to_csv("books_with_emotions.csv", index = False)

In [17]:
print("Available columns:", books.columns.tolist())

Available columns: ['isbn13', 'isbn10', 'title', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count', 'title_and_subtitle', 'tagged_description', 'simple_categories', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
